In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.1`

import $ivy.$                                  

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import java.sql.Date
import scala.collection.JavaConverters._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import java.sql.Date

import scala.collection.JavaConverters._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Hello Spark")
                .getOrCreate()

import spark.implicits._
spark.version

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/07/14 14:18:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/14 14:18:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


spark: SparkSession = org.apache.spark.sql.SparkSession@2886482a
import spark.implicits._

res3_2: String = "3.2.1"

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.2.1


# DataFrame

In [6]:
val df = spark.createDataFrame(Array(
    (1, 2.0, "string1", Date.valueOf("2000-01-01")),
    (2, 3.0, "string2", Date.valueOf("2000-02-01")),
    (3, 4.0, "string3", Date.valueOf("2000-03-01"))
))
df

df: DataFrame = [_1: int, _2: double ... 2 more fields]
res5_1: DataFrame = [_1: int, _2: double ... 2 more fields]

In [7]:
df.printSchema

root
 |-- _1: integer (nullable = false)
 |-- _2: double (nullable = false)
 |-- _3: string (nullable = true)
 |-- _4: date (nullable = true)



In [8]:
val schema = StructType(
    StructField("a", IntegerType, nullable = true) ::
    StructField("b", DoubleType, nullable = true) ::
    StructField("c", StringType, nullable = true) ::
    StructField("d", DateType, nullable = true) ::
    Nil)

schema: StructType = StructType(
  StructField("a", IntegerType, true, {}),
  StructField("b", DoubleType, true, {}),
  StructField("c", StringType, true, {}),
  StructField("d", DateType, true, {})
)

In [8]:
val df1 = spark.createDataFrame(sc.parallelize(
    Seq(
        Row(1, 2.0, "string1", Date.valueOf("2000-01-01")),
        Row(2, 3.0, "string2", Date.valueOf("2000-02-01")),
        Row(3, 4.0, "string3", Date.valueOf("2000-03-01"))
    )),
    schema)
df1

cmd8.sc:1: not found: value sc
val df1 = spark.createDataFrame(sc.parallelize(
                                ^Compilation Failed

: 

In [ ]:
df1.printSchema

In [ ]:
df1.dtypes

In [ ]:
df1.show()

In [ ]:
df1.columns

In [ ]:
df1.select("a", "b", "c").describe().show()

In [ ]:
df1.withColumn("upper_c", upper($"c")).show()

In [ ]:
df1.filter($"a" === 1).show()

In [ ]:
val df2 = spark.createDataFrame(List(
    Row("red", "banana", 1, 10),
    Row("blue", "banana", 2, 20),
    Row("red", "carrot", 3, 30),
    Row("blue", "grape", 4, 40),
    Row("red", "carrot", 5, 50),
    Row("black", "carrot", 6, 60),
    Row("red", "banana", 7, 70),
    Row("red", "grape", 8, 80)).asJava,
StructType(Array(
    StructField("color", StringType, nullable = true),
    StructField("fruit", StringType, nullable = true),
    StructField("v1", IntegerType, nullable = true),
    StructField("v2", IntegerType, nullable = true)))
)

df2.show()

In [ ]:
df2.groupBy("color").avg().show()

In [ ]:
df2.write.mode("overwrite").option("header", "true").csv("foo.csv")
spark.read.option("header", "true").csv("foo.csv").show()

In [ ]:
df.write.mode("overwrite").parquet("bar.parquet")
spark.read.parquet("bar.parquet").show()

In [ ]:
df.write.mode("overwrite").orc("zoo.orc")
spark.read.orc("zoo.orc").show()

In [ ]:
df2.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

In [ ]:
val add_one = (a: String) => a + 1
val add_oneUDF = udf(add_one)

df2.select(add_oneUDF($"v1").as("add_one_v1")).show

In [ ]:
spark.udf.register("add_one", add_oneUDF)
spark.sql("SELECT add_one(v1) FROM tableA").show()